# import

In [2]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from pyvino.model.model import Model
from pyvino.util.config import TASKS
from pyvino.util.image import imshow, cam_test, cv2pil
from pyvino.detector.detector import DetectorHumanPose

https://www.youtube.com/watch?v=Pv6Cimz4-WE

# path

In [3]:
notebook_name = os.path.basename(os.getcwd())
timestamp = notebook_name.split('_')[1]
timestamp

'07'

In [4]:
path_labels_dir = '../data/labels/'
path_process_dir = os.path.join('../data/01_DataProcessing/', timestamp)

In [5]:
path_movie_dir = '../data/movie/fps30_mask/'
path_save_dir = os.path.join('../data/02_GetPoints/', timestamp)
if not os.path.exists(path_save_dir):
    os.makedirs(path_save_dir)

In [6]:
path_labels_name_num = os.path.join(path_labels_dir, 'labels_num.csv')
labels_name_num = pd.read_csv(path_labels_name_num)

In [20]:
# file_names = ['forehand', 'backhand', 'foreslice', 'backslice', 'test']
file_names = ['backslice']
movie_names = [file_name + '.mp4' for file_name in file_names]
points_file_names = [file_name + '.csv' for file_name in file_names]

In [21]:
movie_name = movie_names[0]
path_movie_file = os.path.join(path_movie_dir, movie_name)

# model

In [22]:
TASKS

{'detect_face': 'face-detection-adas-0001',
 'emotion_recognition': 'emotions-recognition-retail-0003',
 'estimate_headpose': 'head-pose-estimation-adas-0001',
 'detect_body': 'person-detection-retail-0013',
 'estimate_humanpose': 'human-pose-estimation-0001',
 'detect_segmentation': 'instance-segmentation-security-0050'}

In [23]:
det_h = DetectorHumanPose()

2019-09-15 12:13:22,185 - pyvino.detector.detector - INFO - human-pose-estimation-0001 on Darwin
2019-09-15 12:13:22,186 - pyvino.detector.detector - INFO - The path to cpu_extension is /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension.dylib
2019-09-15 12:13:22,512 - pyvino.detector.detector - INFO - person-detection-retail-0013 on Darwin
2019-09-15 12:13:22,512 - pyvino.detector.detector - INFO - The path to cpu_extension is /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension.dylib


In [24]:
len(det_h.POSE_PARTS_FLATTEN) - 2

34

In [25]:
body_parts_list = list(det_h.BODY_PARTS.keys())
body_parts_list

['Nose',
 'Neck',
 'RShoulder',
 'RElbow',
 'RWrist',
 'LShoulder',
 'LElbow',
 'LWrist',
 'RHip',
 'RKnee',
 'RAnkle',
 'LHip',
 'LKnee',
 'LAnkle',
 'REye',
 'LEye',
 'REar',
 'LEar']

# movie

In [26]:
cap = cv2.VideoCapture(path_movie_file)

# 幅
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# 高さ
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# 総フレーム数
count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# fps
fps = int(cap.get(cv2.CAP_PROP_FPS))

# compute

In [27]:
canvas_org = np.zeros((height, width, 3), np.uint8)

In [28]:
def add_bbox_margin (xmin, ymin, xmax, ymax, bbox_margin):
    xmin = int(xmin * (1 - bbox_margin))
    ymin = int(ymin * (1 - bbox_margin))
    xmax = int(xmax * (1 + bbox_margin))
    ymax = int(ymax * (1 + bbox_margin))
    return xmin, ymin, xmax, ymax

In [29]:
show = False

In [30]:
points_flattens = np.empty((36, 0))

movie = movies[0]
path_movie_file = os.path.join(path_movie_dir, movie)

In [31]:
mat = np.zeros([1,len(det_h.POSE_PARTS_FLATTEN)])
mat[:,:] = np.nan

In [32]:
for movie in movie_names:
    print(movie)
    movie_name = os.path.splitext(movie)[0]
    path_movie_file = os.path.join(path_movie_dir, movie)
    path_save_points = os.path.join(path_save_dir, movie_name + '.csv')
    
    cap = cv2.VideoCapture(path_movie_file)
    frame_num = 0
    points_flattens = np.empty((0, 36))
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_num % 100 == 0:
            print(frame_num)

        frame = np.array(frame)

        ############################
        height, width, _ = frame.shape

        results = det_h.compute(frame, pred_flag=True, normalize_flag=True)

        if len(results['preds']) == 0:
            points_flattens = np.vstack([points_flattens, mat])
            frame_num += 1
            continue

        area = 0
        for key, values in results['preds'].items():
            xmin, ymin, xmax, ymax = values['bbox']
            area_new = (ymax - ymin) * (xmax - xmin)
            if area_new > area:
                area = area_new
                largest_num = key

        points = results['preds'][largest_num]['norm_points']

        ######
        points_flatten = points.flatten()
        points_flattens = np.vstack([points_flattens, points_flatten])
        ######
        frame_num += 1
        if show:
            cv2.imshow('demo', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    points_df = pd.DataFrame(points_flattens, columns=det_h.POSE_PARTS_FLATTEN)
    points_df.to_csv(path_save_points, index=False)

cap.release()
cv2.destroyAllWindows()

backslice.mp4
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
